# Detecting Wildfires in California

![image](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQkxgiAHnoPLQCy3-Z68vLAWbczrKCDJ37VGV0cKXBUOP9IBNZ8)

### Background

A **wildfire** is an uncontrolled fire in the vegetation of an area, also commonly referred to as a forest fire, bushfire, grassfire, veldfire, etc., depending on type of vegetation burning and/or where you live. California has dry, windy, and often hot weather conditions from spring through late autumn that can produce moderate to devastating wildfires. At times, these wildfires are fanned or made worse by strong, dry winds, known as Diablo winds when they occur in the northern part of the state and Santa Ana winds when they occur in the south. Wildfires in California are growing increasingly dangerous because of climate change and because more people are building in rural burn areas. So far, **in 2019, over 6,402 fires have been recorded** according to Cal Fire and the US Forest Service, totaling an estimated of 250,349 acres (101,313 ha) of burned land as of November 3.

### The Problem

The wildfire once begun, quickly spreads, consuming the thick, dried-out vegetation and almost everything else in its path. What was once a forest becomes a virtual powder keg of untapped fuel. In a seemingly instantaneous burst, the wildfire overtakes thousands of acres of surrounding land, threatening the homes and lives of many in the vicinity. These rolling flames travel up to 14 miles an hour, which converts to about a four-minute mile pace, and can overtake the average human in minutes, and that makes it *important to detect wild fires at the earliest* which helps to regulate them and in turn provides more evacuation time.

In [ ]:
# Ignore some innocuous warnings.
import warnings
import sys
import logging
warnings.filterwarnings('ignore', module='datacube')
logging.basicConfig(stream=sys.stdout, level=logging.CRITICAL)
for logger_name in ('boto', 'boto3', 'botocore', 's3transfer', 'rasterio', 'dask', 
                    'distributed', 'distributed.core', 'distributed.scheduler', 'distributed.client',
                    'distributed.batched', 'urllib3'):
    logging.getLogger(logger_name).setLevel(logging.CRITICAL)
    
import dask
import distributed
from dask_kubernetes import KubeCluster
from dask.distributed import Client
cluster = None
# Click on the 'Dashboard link' to monitor calculation progress
cluster = KubeCluster()
cluster.scale_up(12)
client = distributed.Client(cluster)

In [ ]:
# Import Libraries
import datacube
import datetime
import numpy as np
import panel as pn
import holoviews as hv

sys.path.append('../CEOS Notebooks')
from datacube.storage import masking
import utils.data_cube_utilities.data_access_api as dc_api 
from utils.data_cube_utilities.dc_load import get_overlapping_area
from utils.data_cube_utilities.dc_display_map import display_map 
hv.extension('bokeh')
pn.extension()

In [ ]:
# Connect to Datacube
dc = datacube.Datacube(app="Fire Detection")

### Analysis Parametes

The following cell sets the parameters, which define the area of interest and the length of time to conduct the analysis over.
The parameters are

* `latitude`: The latitude range to analyse `(38.73, 38.83)`.

* `longitude`: The longitude range to analyse `(-122.27, -122.17)`.

* `time_range`: The date range to analyse `('2018, 6, 1', '2018, 10, 1')`.  

In [ ]:
# Set the analysis parameters
platform = 'LANDSAT_8'
product = 'ls8_usgs_scene_digiscape'
area = {'min_lat' : 38.73,
        'max_lat' : 38.83,
        'min_lon' : -122.27,
        'max_lon' : -122.17}
time_range =  {'start_time' : datetime.datetime(2018, 6, 1),
               'end_time' : datetime.datetime(2018, 10, 1)}

In [ ]:
# View the Selected Location
display_map(longitude=(area['min_lon'], area['max_lon']),
            latitude=(area['min_lat'], area['max_lat']))

### Load and view Landsat-8 data

In [ ]:
dask_chunks = {'time':1}

ds = dc.load(
     platform = platform,
     product = product,
     y = (area['min_lat'],
          area['max_lat']),
     x = (area['min_lon'], 
          area['max_lon']),
     crs = 'epsg:4326', # Query coordinate reference system
     output_crs = 'epsg:3488',
     resolution = (-30, 30),
     time = (time_range['start_time'],
             time_range['end_time']),
     dask_chunks = dask_chunks,
     measurements = ['sr_red', 'sr_green', 'sr_blue', 'sr_nir', 'sr_swir2'],
     group_by='solar_day')

ds

In [ ]:
# Set all nodata pixels to `NaN`:
ds = masking.mask_invalid_data(ds)
# Set all invalid data to `NaN` - valid range for USRS SR is 0 to 10000, but the surface reflectance product can have values just outside this range
# We remove them so the image drawn isn't impacted by them
ds = ds.where((ds >= 0) & (ds<=10000))

# Select a time slice from the EO data and combine the bands into a 3 band array
image_array = ds[['sr_red', 'sr_green', 'sr_blue']].isel(time=4).compute().to_array()
# Show the image
image_array.plot.imshow(robust=True, figsize=(10, 10))

### Detecting Wildfires - **NBR**

The Normalized burn ratio (NBR) is used to identify burned areas. This index uses the ratio of the near-infrared (NIR) and shortwave-infrared (SWIR) bands to detect burned areas. The formula is:

$$
\begin{aligned}
\text{NBR} = \frac{\text{NIR} - \text{SWIR}}{\text{NIR} + \text{SWIR}}.
\end{aligned}
$$

Pre-fire, healthy vegetation has very high near-infrared reflectance and low reflectance in the shortwave infrared portion of the spectrum. Recently burned areas on the other hand have relatively low reflectance in the near-infrared and high reflectance in the shortwave infrared band. A high NBR value generally indicates healthy vegetation while a low value indicates bare ground and recently burned areas.

##### Example - [**County Fire**](https://en.wikipedia.org/wiki/County_Fire) (June 30, 2018 – July 17, 2018)

In [ ]:
def intersection(selected_range, prod_range):
    if (selected_range[0] < prod_range[0]) and (selected_range[1] > prod_range[0]):
        min_range, max_range = prod_range[0], min(selected_range[1], prod_range[1])
        
    elif (selected_range[0] < prod_range[1]) and (selected_range[1] > prod_range[1]):
        min_range, max_range = max(selected_range[0], prod_range[0]), prod_range[1]
        
    elif (selected_range[0] > prod_range[0]) and (selected_range[1] < prod_range[1]):
        min_range, max_range = selected_range[0], selected_range[1]
        
    else:
        min_range, max_range = None, None
        
    return min_range, max_range

# Store initial values
prev_value_dict = {'plat' : 'LANDSAT_8',
                   'prod' : 'ls8_usgs_scene_digiscape',
                   'lat_range' : (38.73, 38.83),
                   'lon_range' : (-122.27, -122.17),
                   'time_extent' : (datetime.datetime(2018, 6, 1), datetime.datetime(2018, 10, 1)),
                   'analysis_date' : {'01 Jun 2018': datetime.datetime(2018, 6, 1, 18, 44, 28, 33152)}}

In [ ]:
api = dc_api.DataAccessApi()
dc = api.dc
products_info = dc.list_products()

# Select Widgets
plat_widget = pn.widgets.Select(name='Platform', options=['LANDSAT_8'], value='LANDSAT_8')
prod_widget = pn.widgets.Select(name='Product', options=['ls8_usgs_scene_digiscape'])

full_lat, full_lon, min_max_dates = get_overlapping_area(api, [plat_widget.value], [prod_widget.value])

# Slider Widgets 
lat_widget = pn.widgets.RangeSlider(name='Latitude Range', start=full_lat[0],
                                    end=full_lat[1], value=prev_value_dict['lat_range'], step=0.01)
lon_widget = pn.widgets.RangeSlider(name='Longitude Range', start=full_lon[0],
                                    end=full_lon[1], value=prev_value_dict['lon_range'], step=0.01)

# Time Extent Selection Widget
date_range_slider = pn.widgets.DateRangeSlider(name='Date Range',
                                               start=min_max_dates[0][0], end=min_max_dates[0][1],
                                               value=prev_value_dict['time_extent'])

# Text Widget
lat_text_default = str(prev_value_dict['lat_range'][0]) + ' | ' + str(prev_value_dict['lat_range'][1])
lon_text_default = str(prev_value_dict['lon_range'][0]) + ' | ' + str(prev_value_dict['lon_range'][1])

lat_text = pn.widgets.TextInput(name='Latitude Range ( Min_lat | Max_lat )', value=lat_text_default)
lon_text = pn.widgets.TextInput(name='Longitude Range ( Min_lon | Max_lon )', value=lon_text_default)

# Static Text Widget
static_text = pn.widgets.StaticText(value='')

@pn.depends(plat_widget.param.value, prod_widget.param.value, lat_widget.param.value, 
            lon_widget.param.value, lat_text.param.value, lon_text.param.value,
            date_range_slider.param.value, watch=True)
def update_widget(platform, product, lat, lon, lat_txt, lon_txt, date_range):
    
    # Update product widget
    global lat_intersection, lon_intersection
    prod_list = list(products_info[products_info['platform'] == platform]['name'])
    prod_widget.options = prod_list
    
    if product not in prod_list:
        product = prod_list[0]
        
    prev_value_dict['time_extent'] = date_range
    
    # Update lat and lon widget
    try:
        lat_tup = tuple([float(val) for val in lat_txt.split(' | ')])
        lon_tup = tuple([float(val) for val in lon_txt.split(' | ')])
        assert (len(lat_tup), len(lon_tup)) == (2,2)
    except:
        lat_text.value = str(round(lat[0],2)) + ' | ' + str(round(lat[1],2))
        lon_text.value = str(round(lon[0],2)) + ' | ' + str(round(lon[1],2))
    
    if (len(lat_tup), len(lon_tup))  == (2,2):
        if (lat == prev_value_dict['lat_range']) and (lat_tup != prev_value_dict['lat_range']):
            prev_value_dict['lat_range'] = lat_tup
            lat_widget.value = prev_value_dict['lat_range']

        elif (lat != prev_value_dict['lat_range']) and (lat_tup == prev_value_dict['lat_range']):
            prev_value_dict['lat_range'] = lat
            lat_text.value = str(round(lat[0],2)) + ' | ' + str(round(lat[1],2))

        else:
            None

        if (lon == prev_value_dict['lon_range']) and (lon_tup != prev_value_dict['lon_range']):
            prev_value_dict['lon_range'] = lon_tup
            lon_widget.value = prev_value_dict['lon_range']

        elif (lon != prev_value_dict['lon_range']) and (lon_tup == prev_value_dict['lon_range']):
            prev_value_dict['lon_range'] = lon
            lon_text.value = str(round(lon[0],2)) + ' | ' + str(round(lon[1],2))

        else:
            None
    
    # Update lat, lon, date widget value when the product has no data
    if (platform != prev_value_dict['plat']) or (product != prev_value_dict['prod']):
        
        prev_value_dict['plat'] = platform
        prev_value_dict['prod'] = product
        full_lat, full_lon, min_max_dates = get_overlapping_area(api, [platform], [product])
        
        if list(min_max_dates[0]) != [None, None]:
            lat_widget.start, lat_widget.end, lat_widget.value = full_lat[0], full_lat[1], (prev_value_dict['lat_range'][0],
                                                                                            prev_value_dict['lat_range'][1])
            lon_widget.start, lon_widget.end, lon_widget.value = full_lon[0], full_lon[1], (prev_value_dict['lon_range'][0],
                                                                                            prev_value_dict['lon_range'][1])
            date_range_slider.start, date_range_slider.end, date_range_slider.value = min_max_dates[0][0], min_max_dates[0][1],\
                                                                                      (prev_value_dict['time_extent'][0],
                                                                                       prev_value_dict['time_extent'][1])
            static_text.value = ""
        
        else:          
            static_text.value = "No Data Found for " + product
            
    # Check the intersection of selected lat and lon extents with the products extent
    try:
        full_lat, full_lon, min_max_dates = get_overlapping_area(api, [platform], [product])
        lat_intersection, lon_intersection = intersection(lat, full_lat), intersection(lon, full_lon)
        
        if (lat_intersection == (None, None)) or (lon_intersection == (None, None)):
            if list(min_max_dates[0]) != [None, None]:
                full_lat, full_lon = tuple([round(x,2) for x in full_lat]), tuple([round(x,2) for x in full_lon])
                static_text.value = "There is no data for the selected extents. Use latitude values in the range {}\
                        and longitude values in the range {}".format(full_lat, full_lon)
            
        else:
            lat_intersection = tuple([round(x,2) for x in lat_intersection])
            lon_intersection = tuple([round(x,2) for x in lon_intersection])
            full_lat, full_lon = tuple([round(x,2) for x in full_lat]), tuple([round(x,2) for x in full_lon])
            lat, lon = tuple([round(x,2) for x in lat]), tuple([round(x,2) for x in lon])
            
            if (lat_intersection != lat) and (lon_intersection != lon):
                static_text.value = "The selected latitude and longitude extents are {} and {} but the selected product's latitude\
                                     and longitude extents are {} and {}, so only the latitude {} and longitude range {} will be \
                                     used".format(lat, lon, full_lat, full_lon, lat_intersection, lon_intersection)
            
            elif lat_intersection != lat:
                static_text.value = "The selected latitude extents are {}, but the selected product's latitude extents are {},\
                                     so only the latitude range {} will be used".format(lat, full_lat, lat_intersection)
        
            elif lon_intersection != lon:
                static_text.value = "The selected longitude extents are {}, but the selected product's longitude extents are {},\
                                     so only the longitude range {} will be used".format(lon, full_lon, lon_intersection)
            
            else:
                static_text.value = ""
                
    except:
        static_text.value = ""
        lat_intersection, lon_intersection = None, None
        
pn.Column(pn.Row(plat_widget, prod_widget),
          pn.Row(pn.Row(pn.WidgetBox(lat_widget), pn.WidgetBox(lat_text))),
          pn.Row(pn.Row(pn.WidgetBox(lon_widget), pn.WidgetBox(lon_text))),
          pn.WidgetBox(date_range_slider),
          static_text,
          update_widget,
          width=650, height=350)

In [ ]:
common_load_params = dict(platform = plat_widget.value, product = prod_widget.value, latitude = lat_intersection, longitude = lon_intersection,
                          dask_chunks = {'time':1}, time = date_range_slider.value, measurements = ['sr_red', 'sr_green', 'sr_blue', 'sr_nir', 'sr_swir2'])

def load_data():
    # Load data using Datacube api
    ds = dc.load(**common_load_params, group_by='solar_day')
    
    #Calculate NBR and add it to the loaded dataset
    ds_NBR = (ds.sr_nir-ds.sr_swir2)/(ds.sr_nir+ds.sr_swir2)
    return ds_NBR

ds_NBR = load_data()

In [ ]:
# Date Slider Widget
date_slider_widget = pn.widgets.DiscreteSlider(name = 'Date', options=prev_value_dict['analysis_date'])

# Convert date format from np.datetime64 to datetime.datetime
def date_format_convert(data_cube_time):
    ts = (data_cube_time - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    return datetime.datetime.utcfromtimestamp(ts)

@pn.depends(date_slider_widget.param.value, watch=True)
def get_input_data(analysis_date):
           
    # Store the current slider value
    prev_value_dict['analysis_date'] = {analysis_date.strftime("%d %b %Y") : analysis_date}
    
    # Get the list of scene capture dates in the selected time period
    time_list = [date_format_convert(x) for x in list(ds_NBR.time.values)]
    date_slider_opt = dict(zip([x.strftime("%d %b %Y") for x in time_list], time_list))
    
    # Update Date Slider Options if new data is being loaded
    if date_slider_widget.options != date_slider_opt:
        date_slider_widget.options = date_slider_opt
    
    try:
        index = time_list.index(analysis_date)
    except:
        index = 0

    # Plot NBR data  
    nbr = ds_NBR.isel(time=index).values
    xmin, xmax = float(ds_NBR.longitude.min().values), float(ds_NBR.longitude.max().values)
    ymin, ymax = float(ds_NBR.latitude.min().values), float(ds_NBR.latitude.max().values)
    
    return hv.Image(nbr, bounds=(xmin, ymin, xmax, ymax), label=str(analysis_date.strftime("%d %b %Y"))).opts(colorbar=True, width=600, height=525, 
                                                                                                              cmap='RdYlGn', clim=(-1,1))
    
pn.Column(pn.Row(date_slider_widget),
          pn.Row(get_input_data),
          width=750, height=750)

This notebook gives access to detect the severity of fire over the **County Fire** affected region in California. Red indicates fire affected region and Green indicates healthy vegetated region. Burn severity data and maps can aid in developing emergency rehabilitation and restoration plans - post-fire. They can be used to estimate not only the soil burn severity, but the likelihood of future downstream impacts due to flooding, landslides, and soil erosion.

In [ ]:
client.close()
client = None
cluster.close()
cluster = None

#### -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**References**

1. https://www.nationalgeographic.com/environment/natural-disasters/wildfires/
2. https://science.howstuffworks.com/nature/natural-disasters/wildfire.htm
3. https://en.wikipedia.org/wiki/List_of_California_wildfires
4. https://www.fig.net/resources/proceedings/fig_proceedings/fig2018/ppt/ts05f/TS05F_sabuncu_ozener_9387_ppt.pdf
5. https://en.wikipedia.org/wiki/County_Fire

#### -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------